---
# Stochastic Simulation Assignment 3
---

### **Contributors**  
- **Maarten Stork**  
- **Paul Jungnickel**  
- **Lucas Keijzer**

### **Overview**  
This notebook contains the code and analysis for **Assignment 3 of Stochastic Simulation**. The code follows the order specified in the assignment guidelines and replicates the experiments conducted in the referenced paper. Each section corresponds to (a) key experiment(s).

In [1]:
from CircleParticleSim import *
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

import numpy as np
import numpy.random as rand
import pandas as pd
import matplotlib.pyplot as plt

---

# 1) Optimal Particle Configuration

The following code block generates visual representations of the optimal configurations for \(n = 11, 12,\) and \(50\). These visualizations depict the best possible outcomes and have been included in the report.

In [ ]:
def run_experiment(steps, num_particles, num_runs=10):
    """
    Run the simulation for a given number of particles and store configurations and energies.
    """
    internal_counts = []
    energies = []
    raw_data = []
    energy_examples = {"locations": [], "energies": []}

    for run in range(num_runs):
        sim = CircleParticleSim(
            N=num_particles,
            cooling_schedule=paper_cooling_schedule,
            step_size_schedule=sqrt_step_size_schedule,
            steps=steps
        )
        sim.run_simulation(steps)

        # Calculate floating particles (not on the edge)d
        floating_count = np.sum(np.linalg.norm(sim.particle_locations, axis=1) < 0.99)
        internal_counts.append(floating_count)
        energies.append(sim.E)

        # Store raw data
        raw_data.append({
            "Run": run,
            "Floating Count": floating_count,
            "Energy": sim.E
        })

        # Save configurations and energies
        energy_examples["locations"].append(sim.particle_locations)
        energy_examples["energies"].append(sim.E)

    avg_internal_count = np.mean(internal_counts)
    min_internal_count = np.min(internal_counts)
    max_internal_count = np.max(internal_counts)

    results = {
        "Particles": num_particles,
        "Internal Count (Avg)": avg_internal_count,
        "Internal Count (Min)": min_internal_count,
        "Internal Count (Max)": max_internal_count,
        "Total Energy (Avg)": np.mean(energies)
    }

    print(f"Completed simulation for {num_particles} particles (averaged over {num_runs} runs).")

    return results, raw_data, energy_examples


import numpy as np
import matplotlib.pyplot as plt

def plot_selected_configurations(selected_examples):
    """
    Plot configurations for n=11, n=12, and n=50, each showing the lowest energy configuration.
    """
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 24))  # Adjust layout for vertical alignment
    particle_counts = [11, 12, 50]

    for i, (ax, n_particles) in enumerate(zip(axes, particle_counts)):
        locations = selected_examples[n_particles]["locations"]
        energies = selected_examples[n_particles]["energies"]

        # Find the best configuration (lowest energy)
        best_index = np.argmin(energies)
        best_configuration = locations[best_index]
        best_energy = energies[best_index]

        # Plot the configuration
        thetas = np.linspace(0, 2 * np.pi, 100)
        ax.plot(np.cos(thetas), np.sin(thetas), linestyle='--', color='gray', linewidth=2, alpha=0.7)
        ax.scatter(
            best_configuration[:, 0], best_configuration[:, 1],
            color='blue', edgecolor='black', s=200, alpha=0.9
        )

        # Set the title
        ax.set_title(
            f"n={n_particles}\nLowest Energy: {best_energy:.2f}",
            fontsize=24, fontweight='bold', color='navy', pad=30
        )

        # Styling
        ax.set_xlim([-1.2, 1.2])
        ax.set_ylim([-1.2, 1.2])
        ax.set_aspect('equal')  # Ensure the aspect ratio is equal
        ax.axis('off')

    # Add a global caption
    plt.figtext(
        0.5, 0.01,
        "Selected Optimal Configurations: n=11, n=12, n=50 (lowest energy for each).\n"
        "Simulated annealing used to minimize energy and arrange particles within a circular boundary.",
        ha='center', fontsize=18, color='darkgray', wrap=True
    )

    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.show()


if __name__ == '__main__':
    step_counts = 100000
    num_runs = 50
    particle_counts = [11, 12, 50]

    selected_examples = {}

    # Run experiments for the selected particle counts
    for n in particle_counts:
        print(f"Running experiment for n={n}...")
        results, raw_data, energy_examples = run_experiment(
            steps=step_counts, num_particles=n, num_runs=num_runs
        )
        selected_examples[n] = energy_examples  # Save only the energy examples

    # Plot the selected configurations
    plot_selected_configurations(selected_examples)


Running experiment for n=11...


The following code block runs the model for particle counts ranging from \(n = 9\) to \(n = 50\). It gathers data on particle counts, optimal energies, optimal positional configurations, and the proportional frequency of observing these optimal configurations in the results. A table summarizing these findings has been included in the paper.


In [31]:
def run_experiment(max_particles, steps, num_runs=10):
    """
    Run the simulation for a range of particle numbers and record results.

    Returns:
    - results: A list of dictionaries containing particle count, avg/min/max internal count, energy, and likelihood of optimal configuration.
    - examples: List of tuples containing particle count and their locations.
    - raw_data: Raw data collected for further analysis.
    """
    results = []
    examples = []
    raw_data = []

    for num_particles in range(9, max_particles + 1):
        internal_counts = []
        energies = []
        example_locations = None

        for run in range(num_runs):
            sim = CircleParticleSim(
                N=num_particles,
                cooling_schedule=paper_cooling_schedule,
                step_size_schedule=sqrt_step_size_schedule,
                steps=steps
            )
            sim.run_simulation(steps)

            # Calculate floating particles (not on the edge)
            floating_count = np.sum(np.linalg.norm(sim.particle_locations, axis=1) < 0.99)
            internal_counts.append(floating_count)
            energies.append(sim.E)

            # Save example locations from the first run
            if run == 0:
                example_locations = sim.particle_locations

            # Store raw data
            raw_data.append({
                "Particles": num_particles,
                "Run": run,
                "Floating Count": floating_count,
                "Energy": sim.E
            })

        # Compute metrics
        min_energy = np.min(energies)
        min_energy_index = energies.index(min_energy)
        optimal_floating_count = internal_counts[min_energy_index]

        # Calculate likelihood of achieving the same floating count as the optimal configuration
        optimal_floating_count_matches = np.sum(np.array(internal_counts) == optimal_floating_count)
        optimal_likelihood = optimal_floating_count_matches / num_runs

        avg_internal_count = np.mean(internal_counts)
        min_internal_count = np.min(internal_counts)
        max_internal_count = np.max(internal_counts)
        avg_energy = np.mean(energies)

        # Debugging outputs
        print(f"Energies for {num_particles}: {energies}")
        print(f"Min Energy for {num_particles}: {min_energy}")
        print(f"Optimal Floating Count for {num_particles}: {optimal_floating_count}")
        print(f"Likelihood of Optimal Floating Count: {optimal_likelihood}")

        results.append({
            "Particles": num_particles,
            "Optimal Energy": min_energy,
            "Optimal Floating Particles": optimal_floating_count,
            "Likelihood of Optimal Configuration": optimal_likelihood
        })

        examples.append((num_particles, example_locations))
        print(f"Completed simulation for {num_particles} particles (averaged over {num_runs} runs).")

    return results, examples, raw_data

def create_and_save_node_energy_table(results, output_file):
    """
    Create a DataFrame for particle counts and their optimal metrics, and save it to a CSV file.

    Parameters:
    - results: The results data collected from the simulation.
    - output_file: Path to save the CSV file.
    """
    df = pd.DataFrame(results)
    print("Node Count and Optimal Metrics Table:")
    print(df)

    # Save to a CSV file
    df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    return df

if __name__ == '__main__':
    max_particles = 50
    steps = 100000
    num_runs = 50

    # Run the experiment and collect results
    results, examples, raw_data = run_experiment(max_particles, steps, num_runs)

    # Create and save the node energy table
    output_file = "node_energy_metrics.csv"
    node_energy_df = create_and_save_node_energy_table(results, output_file)


Energies for 9: [59.861408776874896, 59.86873576667551, 59.84796830027177, 59.85060564077723, 60.963588966084075, 60.899578805593954, 60.88864361454442, 60.93412425273214, 60.89785170799189, 60.906893400714125, 60.89631122210763, 59.848055527133155, 60.98048043127547, 60.9016062645329, 59.847482004351136, 60.90854116835488, 60.90377344190435, 60.95442969936869, 60.91446408345836, 60.93692401412224, 60.87848955387011, 60.91145370735868, 60.92335342311508, 60.89547622822858, 60.91427252230952, 59.84737754741198, 59.84779624220255, 60.964540818096935, 60.87995205091655, 60.944673748006494, 59.87715782401953, 60.9239015476484, 61.015830275882294, 60.88552666366799, 60.88384385847206, 60.935544821358214, 59.847668452887945, 59.84724818989044, 60.88199292844735, 60.90007013933191, 60.93044654933604, 59.8775425858572, 59.93338283455358, 59.84739484366085, 60.898753510043036, 60.904040542646925, 60.97895532316031, 60.89311624335649, 60.97065070593238, 60.91251252985758]
Min Energy for 9: 59.84

---
# 2) Cooling Schedules

In [2]:
# Parameters used
num_particles = 12
steps = 10000
num_runs = 50
# markov 100
# cooling 10000


schedules = [
# log_cooling_schedule,
# basic_cooling_schedule,
paper_cooling_schedule,
exponential_cooling_schedule,
# linear_cooling_schedule,
# quadratic_cooling_schedule,
sigmoid_cooling_schedule,
inverse_sqrt_cooling_schedule,
cosine_annealing_cooling_schedule,
# stepwise_cooling_schedule,
]

In [ ]:
data = {}

# generate data
for schedule in schedules:
    print(f"Currently running for: {schedule.__name__}")
    mean_energy, std_energy, mean_temperatures = evaluate_multiple_runs(
        num_particles, cooling_schedule=schedule, steps=steps, num_runs=num_runs
    )
    data[schedule.__name__] = {
        "mean_energy": mean_energy,
        "std_energy": std_energy,
        "mean_temperatures": mean_temperatures
    }

# Save data
np.save('data/data-2-{}-{}.npy'.format(num_particles, len(schedules)), data)


In [ ]:
loaded_data = np.load('data/data-2-{}-{}.npy'.format(num_particles, len(schedules)), allow_pickle=True).item()

fig, axs = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

axins = inset_axes(
    axs[1],
    width="30%", 
    height="50%", 
    loc='upper right'
)

for schedule_name, results in loaded_data.items():
    mean_energy = results["mean_energy"]
    std_energy = results["std_energy"]
    mean_temperatures = results["mean_temperatures"]
    print(f"min energy: {min(mean_energy)} for schedule: {schedule_name}")
    axs[1].plot(mean_energy, label=schedule_name)
    axs[1].fill_between(
        range(len(mean_energy)),
        mean_energy - std_energy,
        mean_energy + std_energy,
        alpha=0.3
    )
    axs[0].plot(mean_temperatures, label=schedule_name)
    
    axins.plot(mean_energy, label=schedule_name)
    axins.fill_between(
        range(len(mean_energy)),
        mean_energy - std_energy,
        mean_energy + std_energy,
        alpha=0.3
    )

# Plot for temperature
axs[0].set_xlabel("Steps")
axs[0].set_ylabel("Temperature")
axs[0].set_title("Temperature Evolution Over Time")
axs[0].set_xscale('log')
# axs[1].set_yscale('log')
axs[0].legend()
axs[0].grid(True)

# Plot for energy
axs[1].set_ylabel("Energy")
axs[1].set_title("Mean Energy with Standard Deviation Over Time")
axs[1].set_xscale('log')
# axs[1].set_yscale('log')
# axs[1].legend()
axs[1].grid(True)


skip_first_steps = 0
plt.xlim(left=skip_first_steps)

# zoomed in plot
axins.set_xlim(10**2, 10**4)
axins.set_ylim(119, 121)
axins.set_xscale('log')
axins.set_yscale('log')
axins.grid(True)

mark_inset(axs[1], axins, loc1=2, loc2=4, fc="none", ec="0.5")




plt.tight_layout()
plt.show()

---
# 3) TITLE??

In [ ]:
rand.seed(42)
run_count = 100
num_scales = 50
num_particles = 50
scales = np.logspace(-2,1,num=num_scales, base=10)
print(scales)
arrs = np.zeros([num_scales, run_count]) 

for i, scale in enumerate(scales):
    num_steps = int(100000 * scale)
    for j in range(run_count):
        sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
                    cooling_schedule = paper_cooling_schedule,
                    step_size_schedule = sqrt_step_size_schedule,
                    random_step_likelihood=0.2,
                    extra_args = {'cooling_schedule_scaling' : scale} 
                    )
        sim.run_simulation(num_steps)
        arrs[i,j] = sim.E

    print('.', end='')

    
np.save('data/data-3-{}-{}.npy'.format(num_particles, num_scales), arrs)

In [ ]:

arrs = np.load('data/data-3-50-50.npy')
arrs = arrs[:]
scales = np.logspace(-2,1,num=num_scales, base=10)
scales1 = 100*scales[:]
print(arrs.shape)
mean = np.mean(arrs, axis=1)
min_energy = np.min(arrs)
plt.plot(scales1, mean, label='mean energy + IQR')
# plt.plot(probs, np.percentile(arrs, 5, axis=1))
plt.fill_between(scales1, np.percentile(arrs, 0, axis=1), np.percentile(arrs, 99, axis=1), alpha=0.3)
plt.plot(scales1, min_energy*np.ones_like(scales1), linestyle = ':', color='gray', label='lowest measured energy')
plt.xlabel('chain length scaling')
plt.ylabel('E')
plt.xscale('log')
plt.ylim([2900, 2950])
plt.legend()

---

# 4) TITLE??

In [ ]:
rand.seed(42)
run_count = 100
num_probs = 50
num_particles = 16
probs = np.linspace(0,1,num_probs)
arrs = np.zeros([num_probs, run_count]) 

for i, p in enumerate(probs):
    for j in range(run_count):
        sim  = CircleParticleSim(num_particles, steps=100000, seed=rand.randint(0,2**31-1),
                    cooling_schedule = paper_cooling_schedule,
                    step_size_schedule = sqrt_step_size_schedule,
                    random_step_likelihood=p
                    )
        sim.run_simulation(10000)
        arrs[i,j] = sim.E

    print('.', end='')

    
np.save('data/data-4-{}-{}.npy'.format(num_particles, num_probs), arrs)





In [ ]:

arrs = np.load('data/data-4-50-50.npy')
arrs = arrs[2:]
num_probs = 20
probs = np.linspace(0,1,num_probs)
probs1 = probs[2:]
print(arrs.shape)
mean = np.mean(arrs, axis=1)
min_energy = np.min(arrs)
plt.plot(probs1, mean, label='mean energy + IQR')
# plt.plot(probs, np.percentile(arrs, 5, axis=1))
plt.fill_between(probs1, np.percentile(arrs, 0, axis=1), np.percentile(arrs, 99, axis=1), alpha=0.3)
plt.plot(probs1, min_energy*np.ones_like(probs1), linestyle = ':', color='gray', label='lowest measured energy')
plt.xlabel('P(random step)')
plt.ylabel('E')
plt.legend()